In [2]:
# Import necessary libraries
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from api_credentials import client_id, client_secret, redirect_uri

# Function to fetch playlist tracks and return as a DataFrame
def get_playlist_tracks(client_id, client_secret, redirect_uri, playlist_id):
    scope = 'playlist-read-private'
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                                   client_secret=client_secret,
                                                   redirect_uri=redirect_uri,
                                                   scope=scope))
    
    tracks_data = []
    
    # Handle pagination
    results = sp.playlist_tracks(playlist_id)
    while results:
        for item in results['items']:
            # Check if the track is not None
            if item['track'] is not None:
                track = item['track']
                # Additional check for track's album and its id
                if track['album'] is not None and 'id' in track['album']:
                    album_id = track['album']['id']
                    album_details = sp.album(album_id)

                    track_info = {
                        'Artist': ', '.join([artist['name'] for artist in track['artists']]),
                        'Song Title': track['name'],
                        'Popularity Score': track['popularity'],
                        'Record Label': album_details['label'] if 'label' in album_details else 'Not Available'
                    }
                    tracks_data.append(track_info)

        # Fetch the next page of tracks
        results = sp.next(results) if results['next'] else None

    df = pd.DataFrame(tracks_data)
    df.index = df.index + 1
    return df


# Mapping of string names to playlist IDs
playlist_names = {
    # top 10 global playlists 
    # 'todays_top_hits' : "37i9dQZF1DXcBWIGoYBM5M",
    # 'top_50_global' : "37i9dQZEVXbMDoHDwVN2tF",
    # 'rap_caviar' : "37i9dQZF1DX0XUsuxWHRQd",
    # 'viva_latino' : "37i9dQZF1DX10zKzsJ2jva",
    # 'rock_classics' : "37i9dQZF1DWXRqgorJj26U",
    # 'baila_reggaeton' : "37i9dQZF1DWY7IeIP1cdjF",
    # 'all_out_2000s' : "37i9dQZF1DX4o1oenSJRJd",
    # 'songs_to_sing_in_the_car' : "37i9dQZF1DWWMOmoXKqHTD",
    # 'all_out_80s' : "37i9dQZF1DX4UtSsGT1Sbe",
    # 'beast_mode': "37i9dQZF1DX76Wlfdnj7AP",

    # # top AU local playlist (not in order, just a sample)
    'NMF_df': '37i9dQZF1DWT2SPAYawYcO',
    # 'HHA_df': '37i9dQZF1DWXXs9GFYnvLB',
    # 'FL_df': '37i9dQZF1DX5WTH49Vcnqp',
    # 'A1_df': '37i9dQZF1DX0sDai2F5jCQ',
    # 'DG_df': '37i9dQZF1DX1lp03JVa0o6',
    # 'GP_df': '37i9dQZF1DXboDblu798Pk'
}

# Dictionary to store each playlist DataFrame
playlists_data = {}

# Process each playlist and store the DataFrame in the dictionary
for name, pid in playlist_names.items():
    playlists_data[name] = get_playlist_tracks(client_id, client_secret, redirect_uri, pid)

# Rename columns for each DataFrame in playlists_data
for df in playlists_data.values():
    df.columns = ['artist', 'song_title', 'popularity_score', 'record_label']

# assign playlist dictionary data to standalone dataframes for each playlist 
nmf_df = playlists_data['NMF_df']
# hha_df = playlists_data['HHA_df']
# fl_df = playlists_data['FL_df']
# a1_df = playlists_data['A1_df']
# dancegen_df = playlists_data['DG_df']
# getpopped_df = playlists_data['GP_df'] 

#global
# todays_top_hits = playlists_data['todays_top_hits']
# top_50_global = playlists_data['top_50_global']
# rap_caviar = playlists_data['rap_caviar']
# viva_latino = playlists_data['viva_latino'],
# rock_classics = playlists_data['rock_classics'],
# baila_reggaeton = playlists_data['baila_reggaeton'],
# all_out_2000s = playlists_data['all_out_2000s'],
# songs_to_sing_in_the_car = playlists_data['songs_to_sing_in_the_car']
# all_out_80s = playlists_data['all_out_80s'],
# beast_mode = playlists_data['beast_mode'],
 

The below are new songs with the top ten `popularity score` after NMF release two days ago. 
Will these be added to 'Singled Out' playlist? 

In [16]:
best_performing = nmf_df[['artist', 'song_title', 'popularity_score']].sort_values(by='popularity_score', ascending=False).head(10)
best_performing


,artist,song_title,popularity_score
5,"Sia, Kylie Minogue",Dance Alone,66
13,Kacey Musgraves,Deeper Well,62
32,"Luciano, Omah Lay",Another Vibe (feat. Omah Lay),62
7,Maggie Rogers,Don't Forget Me,59
34,"Caroline Polachek, Weyes Blood",Butterfly Net (feat. Weyes Blood),57
60,"Chief Keef, Mike WiLL Made-It",DIRTY NACHOS,55
25,Jennifer Lopez,This Is Me…Now,53
62,Faye Webster,Feeling Good Today,49
44,Beth Gibbons,Floating On A Moment,46
54,Lay Bankz,Tell Ur Girlfriend,46


### Hopefully get a larger unique labels list from adding playlists. Ensure all UMG labels are covered for proper analysis. 

In [3]:
# List to hold unique record labels from all dataframes
unique_labels = []

# Iterate over each dataframe in the dictionary
for df_name, df in playlists_data.items():
    # Check if 'record_label' column exists in the dataframe
    if 'record_label' in df.columns:
        # Extract unique labels and add them to the list
        unique_labels.extend(df['record_label'].unique())

# Remove duplicates and create a final list of unique record labels
final_unique_labels = list(set(unique_labels))

# If you want to sort the final list
final_unique_labels.sort()

print(len(final_unique_labels))


224


In [4]:
print(final_unique_labels)

["'94 Sounds/RCA Records", '046 Records', '10K Projects/Capitol Records', '300 Entertainment', '3Beat', '4 of clubs LLC', '4AD', '777', '88rising Music', '@prodbyhasin', 'A$AP Worldwide/RCA Records', 'ABC Music', 'ACTS Music Group', 'AETERNA Records', 'Adrey', 'After Platinum LLC', 'Arcadia Records', 'Arista Records', 'Armada Music', 'Artemas', 'Artist Partner Group, Inc.', 'Astral People Recordings', 'Atlantic / Warner Music Australia', 'Atlantic Records', 'Atlantic Records UK', 'Audiotree Music', 'August Avenue Limited', 'BIGHIT MUSIC', 'BIORDI', 'BMG Rights Management (Australia) Pty Ltd.', 'BMG Rights Management (UK) Ltd.', 'BMG Rights Management (US) LLC', 'Ballona Record Company', 'Because Music', 'Big Loud Records / Mercury Records / Republic Records', 'Big Noise Music Group', 'Biordi Music', 'Black Butter', 'Black Caviar', 'Blake Rose', 'Brainfeeder', 'CHROMA', 'CLUBWRK', 'CXLOE', 'Cactus Jack / Atlantic', 'Cactus Jack/Epic', 'Capitol Records', 'Central Station Records', 'Chaos

In [9]:
# todays_top_hits['record_label'].unique().tolist()

['Generation Now/Atlantic',
 'Republic Records',
 'Top Dawg Entertainment/RCA Records',
 'Geffen Records',
 'RCA Records Label',
 'XO / Republic Records',
 'Warner Records',
 'Kemosabe Records/RCA Records',
 'Slaughter Gang, LLC/Epic Records',
 'FAX Records/Epic',
 'Mercury Records/Republic Records',
 'Taylor Swift',
 'Warner Records/Major Recordings',
 'OVO/Republic',
 'Island Records',
 'Darkroom/Interscope Records',
 "'94 Sounds/RCA Records",
 'Dreamville/Interscope',
 'Olivia Rodrigo PS',
 'Night Street Records, Inc./Warner Records Inc.',
 'Dead Oceans',
 'Mavin Records / Jonzing World',
 'DALE PLAY Records',
 'Hot Girl Productions',
 'Atlantic Records',
 'BIGHIT MUSIC',
 'Double P Records',
 'Interscope Records',
 'Lions Gate Entertainment Inc. under exclusive license to Geffen Records',
 'Ministry of Sound Recordings',
 '10K Projects/Capitol Records',
 'Kygo/RCA Records',
 'Rimas Entertainment LLC',
 'ADOR',
 'UMLE - Latino',
 '300 Entertainment',
 'Cactus Jack/Epic',
 'Columbia'

In [5]:
# List of record labels to keep
labels_to_keep = ["Universal", "Interscope", "Polydor", "Mercury", "Republic", "Capitol", "Island", "Decca", "Def Jam", "Taylor Swift", "Olivia Rodrigo", "UMC (Universal Music Catalogue)", "UME - Global Clearing House", "UMLE - Latino", "UMe Direct 2", "UNI/MOTOWN", "UNIDISC MUSIC INC.", "Roc Nation"]
pattern = '|'.join(labels_to_keep)


# Dictionary to store filtered and sorted DataFrames
filtered_playlists_data = {}

# Process each playlist DataFrame
for name, df in playlists_data.items():
    # Filter the DataFrame based on record label
    filtered_df = df[df['record_label'].str.contains(pattern, case=False, na=False)]

    # Sort the DataFrame by index
    filtered_df_sorted = filtered_df.sort_index()

    # Store the sorted DataFrame
    filtered_playlists_data[name] = filtered_df_sorted

    # Optionally, print out some stats
    print(f"{name}: UMG has {len(filtered_df_sorted)} out of {len(df)} songs")

# Now, each DataFrame in filtered_playlists_data is filtered and sorted

NMF_df: UMG has 8 out of 70 songs
HHA_df: UMG has 12 out of 50 songs
FL_df: UMG has 12 out of 80 songs
A1_df: UMG has 16 out of 62 songs
DG_df: UMG has 8 out of 80 songs
GP_df: UMG has 16 out of 79 songs


/var/folders/px/x65fp4pn31s5cv4gsbh_3cl80000gn/T/ipykernel_13541/1903073965.py:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df = df[df['record_label'].str.contains(pattern, case=False, na=False)]


## New Music Friday - UMG tracks & positions 

In [6]:
# checking the UMG tracks in New Music Friday AU & NZ
filtered_playlists_data['NMF_df']

,artist,song_title,popularity_score,record_label
2,Noah Kahan,Forever,0,Mercury Records/Republic Records
6,Maggie Rogers,Don't Forget Me,0,Capitol Records
12,Kacey Musgraves,Deeper Well,0,Interscope Records/MCA Nashville
14,"James Hype, Kim Petras",Drums (Feat. Kim Petras),78,Universal-Island Records Ltd.
15,Em Beihold,Maybe Life Is Good,0,Republic Records
22,Conan Gray,Lonely Dancers,0,Republic Records
39,Dylan,The Alibi,0,Universal-Island Records Ltd.
50,"Gotye, FISHER, Chris Lake, Kimbra, Sante Sansone",Somebody (2024),0,Universal-Island Records Ltd.


## Hot Hits Australia - UMG tracks & positions 

In [7]:
filtered_playlists_data['HHA_df']

,artist,song_title,popularity_score,record_label
5,Noah Kahan,Stick Season,97,Mercury Records/Republic Records
8,Ariana Grande,"yes, and?",97,Republic Records
15,Sophie Ellis-Bextor,Murder On The Dancefloor,94,Polydor Records
17,Sabrina Carpenter,Feather,93,Island Records
21,Taylor Swift,Cruel Summer,99,Taylor Swift
24,"JID, 21 Savage, Baby Tate",Surround Sound (feat. 21 Savage & Baby Tate),94,Dreamville/Interscope
28,Post Malone,Chemical,87,Mercury Records/Republic Records
38,"Drake, Sexyy Red, SZA",Rich Baby Daddy (feat. Sexyy Red & SZA),93,OVO/Republic
43,Olivia Rodrigo,vampire,95,Olivia Rodrigo PS
48,Nicki Minaj,FTCU,90,Republic Records


## Front Left - UMG tracks & positions

In [8]:
filtered_playlists_data['FL_df']

,artist,song_title,popularity_score,record_label
3,The Last Dinner Party,The Feminine Urge,64,Universal-Island Records Ltd.
5,"Noah Kahan, Sam Fender",Homesick (with Sam Fender),83,Mercury Records/Republic Records
6,"San Cisco, Nicholas Allbrook",Summer Days,45,Island City Records
9,Lola Young,Wish You Were Dead,63,Universal-Island Records Ltd.
18,Noah Kahan,Forever,0,Mercury Records/Republic Records
33,Matt Corby,Desert Land,47,Universal Music Australia Pty. Ltd.
36,Still Woozy,Anyone But You,69,"Still Woozy Productions Inc., under exclusive ..."
42,"Mumford & Sons, Pharrell Williams",Good People,71,Universal-Island Records Ltd.
45,The Last Dinner Party,Caesar on a TV Screen,68,Universal-Island Records Ltd.
52,Lana Del Rey,"Take Me Home, Country Roads",78,Polydor Records


## A1 - UMG tracks & positions

In [9]:
filtered_playlists_data['A1_df']

,artist,song_title,popularity_score,record_label
3,Ice Spice,Think U The Shit (Fart),84,10K Projects/Capitol Records
10,"Lyrical Lemonade, Lil Tecca, The Kid LAROI, Li...","This My Life (with Lil Tecca, The Kid LAROI & ...",75,Lyrical Lemonade P&D/ Def Jam
16,Kobie Dee,Statistic,47,Universal Music Australia Pty. Ltd.
18,"JID, 21 Savage, Baby Tate",Surround Sound (feat. 21 Savage & Baby Tate),94,Dreamville/Interscope
20,"Young Franco, S1mba",HITA,51,Universal Music Australia Pty. Ltd.
23,Nicki Minaj,FTCU,90,Republic Records
24,"Coi Leray, Mike WiLL Made-It",Wanna Come Thru,66,Coi Leray PS/ Island
30,"Drake, J. Cole",Evil Ways (feat. J. Cole),74,OVO/Republic
32,"Nicki Minaj, J. Cole",Let Me Calm Down (feat. J. Cole),75,Republic Records
38,"Bas, J. Cole",Home Alone (with J. Cole),69,Dreamville/Interscope


## Dance Generation - UMG tracks & positions

In [10]:
filtered_playlists_data['DG_df']

,artist,song_title,popularity_score,record_label
6,Lance Savali,we don't sleep,49,Universal Music New Zealand Limited
11,"Gotye, FISHER, Chris Lake, Kimbra, Sante Sansone",Somebody (2024),0,Universal-Island Records Ltd.
21,"Becky Hill, Sonny Fodera",Never Be Alone,74,Polydor Records
22,"James Hype, Kim Petras",Drums (Feat. Kim Petras),78,Universal-Island Records Ltd.
27,"Kungs, David Guetta, Izzy Bizu",All Night Long,78,Universal Music Division Island Def Jam
29,"Jax Jones, Zoe Wees",Never Be Lonely,0,Polydor Records
48,"Jax Jones, RobbieG",Forever,61,Polydor Records
67,"Skepta, Amy Winehouse",Can't Play Myself (A Tribute To Amy),68,Universal-Island Records Ltd.


## Get Popped! - UMG tracks & positions

In [11]:
filtered_playlists_data['GP_df']

,artist,song_title,popularity_score,record_label
1,Sophie Ellis-Bextor,Murder On The Dancefloor,94,Polydor Records
2,Ariana Grande,"yes, and?",97,Republic Records
9,Taylor Swift,Cruel Summer,99,Taylor Swift
10,Sabrina Carpenter,Feather,93,Island Records
17,"The Weeknd, Playboi Carti, Madonna",Popular (with Playboi Carti & Madonna) - From ...,96,XO / Republic Records
20,"Becky Hill, Sonny Fodera",Never Be Alone,74,Polydor Records
21,"Reneé Rapp, Megan Thee Stallion",Not My Fault (with Megan Thee Stallion),81,Paramount Pictures/Interscope Records
27,Post Malone,Chemical,87,Mercury Records/Republic Records
28,Dean Lewis,Memories,71,Universal Music Australia Pty. Ltd.
41,"Drake, Sexyy Red, SZA",Rich Baby Daddy (feat. Sexyy Red & SZA),93,OVO/Republic


In [14]:
# filtered_playlists_data['todays_top_hits']

In [ ]:
filtered_playlists_data['baila_reggaeton']

,artist,song_title,popularity_score,record_label
4,KAROL G,OKI DOKI,81,Bichota/Interscope Records
6,"Gaby Music, Lunay, Luar La L",No Te Quieren Conmigo,88,"Universal Music Latino / Gaby Music, Inc."
9,"Feid, ATL Jacob",LUNA,97,UMLE - Latino
13,"Feid, Young Miko",CLASSY 101,87,UMLE - Latino
14,Bad Gyal,Perdió Este Culo,70,Universal Music Latino/Interscope
15,"KAROL G, Cris Mj, Ryan Castro",UNA NOCHE EN MEDELLÍN - REMIX,79,Bichota/Interscope Records
19,"KAROL G, Peso Pluma",QLONA,93,Bichota/Interscope Records
20,"Feid, Pirlo",CUAL ES ESA,83,UMLE - Latino
21,"Brray, Anuel AA, Chencho Corleone, Jhayco, Rya...",Corazón Roto pt. 3,84,UMLE - Latino
23,"Bad Gyal, Tokischa, Young Miko",Chulo pt.2,87,Universal Music Latino/Interscope


In [ ]:
filtered_playlists_data.keys()

dict_keys(['todays_top_hits', 'top_50_global', 'rap_caviar', 'viva_latino', 'rock_classics', 'baila_reggaeton', 'all_out_2000s', 'songs_to_sing_in_the_car', 'all_out_80s', 'beast_mode', 'NMF_df', 'HHA_df', 'FL_df', 'A1_df', 'DG_df', 'GP_df'])

In [ ]:
# Loop through each key (playlist name) in the dictionary
for playlist_name, df in filtered_playlists_data.items():
    # Save each dataframe to a CSV file
    # The file name is the playlist name with .csv extension
    df.to_csv(f"{playlist_name}.csv", index=False)


In [ ]:
filtered_playlists_data['todays_top_hits']

,artist,song_title,popularity_score,record_label
1,Ariana Grande,"yes, and?",97,Republic Records
4,"The Weeknd, JENNIE, Lily-Rose Depp","One Of The Girls (with JENNIE, Lily Rose Depp)",98,XO / Republic Records
10,Noah Kahan,Stick Season,97,Mercury Records/Republic Records
11,Taylor Swift,Cruel Summer,99,Taylor Swift
14,"Drake, Sexyy Red, SZA",Rich Baby Daddy (feat. Sexyy Red & SZA),93,OVO/Republic
15,Sabrina Carpenter,Feather,93,Island Records
16,Billie Eilish,What Was I Made For? [From The Motion Picture ...,95,Darkroom/Interscope Records
18,Olivia Rodrigo,vampire,95,Olivia Rodrigo PS
20,"JID, 21 Savage, Baby Tate",Surround Sound (feat. 21 Savage & Baby Tate),94,Dreamville/Interscope
28,"The Weeknd, Playboi Carti, Madonna",Popular (with Playboi Carti & Madonna) - From ...,96,XO / Republic Records
